# **ML Mildew Detector**

## Objectives

* Fetch data from Kaggle and save as raw data
* Inspect the data and save it under outputs/datasets/collection

## Inputs

* Kaggle JSON file - the authentication token

## Outputs

* Generate Dataset: inputs/datasets/mildew_dataset

## Additional Comments

* The dataset contains +4 thousand images taken from the client's crop fields. The images show healthy cherry leaves and cherry leaves that have powdery mildew, a fungal disease that affects many plant species.
* The client provided the data under an NDA (non-disclosure agreement), therefore the data should only be shared with professionals that are officially involved in the project.

---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\All\\Desktop\\Zoe_Deve\\ML-Mildew-Detector-\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\All\\Desktop\\Zoe_Deve\\ML-Mildew-Detector-'

# Fetch data from Kaggle

Fetch data from Kagggle website and download the zip file to the folder created at below

In [6]:
import os
try:
  os.makedirs(name='inputs/mildew_dataset') # create inputs/mildew_dataset folder
except Exception as e:
  print(e)   

Unzip the downloaded file, and delete the zip file.

In [7]:
import zipfile
with zipfile.ZipFile('inputs/mildew_dataset' + '/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('inputs/mildew_dataset')

os.remove('inputs/mildew_dataset' + '/archive.zip')

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
